In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.contrib import rnn
import tensorflow.contrib.legacy_seq2seq as seq2seq

In [2]:
nodes_per_layer = 100
layers = 3
dropout_keep_prob = 0.99
batch_size = 100
seq_length = 25
embedding_size = 10
corpus_symbols=30

In [5]:
sess =  tf.Session()
cells = [rnn.GRUCell(nodes_per_layer) for _ in  range(layers)]
#dropout_cells = [rnn.DropoutWrapper(cell,dropout_keep_prob) for cell in cells]
cell = rnn.MultiRNNCell(cells, state_is_tuple=True)

input_data = [tf.placeholder(tf.int32, [1]) for x in range(seq_length)]
targets = [tf.placeholder(tf.int32, [1]) for x in range(seq_length)]
initial_state = cell.zero_state(batch_size, tf.float32)

outputs, states = seq2seq.embedding_rnn_seq2seq(
    input_data, targets, cell,
    corpus_symbols, corpus_symbols,
    embedding_size, output_projection=None,
    feed_previous=False)

loss = seq2seq.sequence_loss_by_example(outputs, targets)
optimizer = tf.train.AdagradOptimizer(0.02)
train_op = optimizer.minimize(loss)

ValueError: Variable embedding_rnn_seq2seq/rnn/embedding_wrapper/embedding already exists, disallowed. Did you mean to set reuse=True in VarScope? Originally defined at:

  File "/home/chris/miniconda3/lib/python3.6/site-packages/tensorflow/contrib/rnn/python/ops/core_rnn_cell_impl.py", line 878, in __call__
    dtype=data_type)
  File "/home/chris/miniconda3/lib/python3.6/site-packages/tensorflow/contrib/rnn/python/ops/core_rnn.py", line 184, in <lambda>
    call_cell = lambda: cell(input_, state)
  File "/home/chris/miniconda3/lib/python3.6/site-packages/tensorflow/contrib/rnn/python/ops/core_rnn.py", line 197, in static_rnn
    (output, state) = call_cell()
